# Homework 3 Solutions

1. Download daily price data for QQQ, TLT, GLD, RWO from yahoo finance since 2016-01-01. Using the adjusted close price data, compute daily returns. This should be a DataFrame with index=date, columns=ticker and values=daily returns.

In [1]:
import yfinance as yf

In [2]:
univ = ['QQQ','TLT','GLD','RWO']
px = yf.download(univ, start="2016-01-01")

[*********************100%***********************]  4 of 4 completed


In [3]:
adj_close = px['Adj Close']
ret = adj_close / adj_close.shift() - 1

In [4]:
ret

Ticker,GLD,QQQ,RWO,TLT
Date,,,,
2016-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN
2016-01-05 00:00:00+00:00,0.002819,-0.001735,0.012116,-0.004034
2016-01-06 00:00:00+00:00,0.014441,-0.009606,-0.007696,0.013476
2016-01-07 00:00:00+00:00,0.014140,-0.031314,-0.017880,0.001794
2016-01-08 00:00:00+00:00,-0.004428,-0.008201,-0.015135,0.004479
...,...,...,...,...
2024-10-14 00:00:00+00:00,-0.001630,0.008391,0.003032,0.000747
2024-10-15 00:00:00+00:00,0.003468,-0.013367,0.006045,0.012157
2024-10-16 00:00:00+00:00,0.005002,0.000122,0.011159,0.004214


2. Compute the signal. Complete the compute_momentum function which computes a simple momentum signal. The function takes in a DataFrame with index = date, columns=ticker and values containing daily returns. It returns a new DataFrame with index = date, columns=ticker and values containing the momentum signal for the ticker on that day. The momentum signal for each ticker is defined as the annualized sharpe ratio of the past 252 days.

In [5]:
import math

def compute_momentum(ret):
    momentum = ret.rolling(252).mean()/ret.rolling(252).std()*math.sqrt(252)
    return momentum

momentum = compute_momentum(ret)
momentum

Ticker,GLD,QQQ,RWO,TLT
Date,,,,
2016-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN
2016-01-05 00:00:00+00:00,NaN,NaN,NaN,NaN
2016-01-06 00:00:00+00:00,NaN,NaN,NaN,NaN
2016-01-07 00:00:00+00:00,NaN,NaN,NaN,NaN
2016-01-08 00:00:00+00:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2024-10-14 00:00:00+00:00,2.475150,1.813120,1.508452,0.879247
2024-10-15 00:00:00+00:00,2.330061,1.807920,1.576620,0.845391
2024-10-16 00:00:00+00:00,2.401506,1.747159,1.587333,0.981408


3. Create a portfolio. Complete the function compute_portfolio. This function takes as input the DataFrame "momentum" from above. It returns a new DataFrame "portfolio" which has the same index/columns and has as values portfolio weights. The weights are computed as follows. On each date, equal-weight long the tickers with a momentum signal above 1.

In [7]:
def compute_portfolio(momentum):
    portfolio = (momentum > 1)*1
    portfolio = portfolio.div(portfolio.abs().sum(1),0)
    return portfolio

portfolio = compute_portfolio(momentum)
portfolio

Ticker,GLD,QQQ,RWO,TLT
Date,,,,
2016-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN
2016-01-05 00:00:00+00:00,NaN,NaN,NaN,NaN
2016-01-06 00:00:00+00:00,NaN,NaN,NaN,NaN
2016-01-07 00:00:00+00:00,NaN,NaN,NaN,NaN
2016-01-08 00:00:00+00:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2024-10-14 00:00:00+00:00,0.333333,0.333333,0.333333,0.00
2024-10-15 00:00:00+00:00,0.333333,0.333333,0.333333,0.00
2024-10-16 00:00:00+00:00,0.333333,0.333333,0.333333,0.00


4. Portfolio returns. Using the "portfolio" returned in part(3) and the returns generated in part(1), compute the returns to the simple momentum strategy. 
- What is the annualized sharpe ratio of the strategy? 
- How about the annualized sharpe ratio within each year? 
- How correlated is the strategy with the underlying tickers?
- Plot the cumulative sum of the returns through time

In [8]:
strat_ret = (portfolio.shift()*ret).sum(1)
# minor, but should start returns when signal starts
strat_ret = strat_ret.loc[momentum.dropna().index[0]:] 

In [ ]:
# sharpe
strat_ret.mean()/strat_ret.std()*math.sqrt(252)

In [ ]:
# sharpe within each year
import numpy as np 

sharpe = lambda x: x.mean()/x.std()*math.sqrt(252) 
strat_ret.groupby([x.year for x in strat_ret.index]).apply(sharpe)

In [ ]:
# correlation
ret.corrwith(strat_ret)

In [ ]:
# cumulative returns
strat_ret.cumsum().plot()